In [21]:
import pulp
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import itertools

In [23]:
# Working through the hooks puzzles.
# Solving the matrix is fast using the same tools as the Feb 14 puzzle 
# but I'm looping through all possible arrangements of hooks which is slow

url='https://www.janestreet.com/puzzles/hooks-2/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
x =[text for text in soup.body.stripped_strings]

print(" ".join(x[7:]))

The grid below can be partitioned into 9 L-shaped “hooks”.  The largest is 9-by-9 (contains 17 squares), the next largest is 8-by-8 (contains 15 squares), and so on.  The smallest hook is just a single square. Find where the hooks are located, and place nine 9’s in the largest hook, eight 8’s in the next-largest, etc., down to one 1 in the smallest hook. The goal is for the sum of the numbers in each row and column to match the number given outside the grid. As your answer to this puzzle, submit the largest product one can achieve using a subset of the numbered squares in the completed grid, satisfying the condition that no two squares in the subset are in the same row or column.


### Puzzle details
<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2016/05/may16_puzzle_expanded.png" width="500" height="600">

In [3]:
#Setup the constraints
col_labels = [45,44,4, 48, 7, 14, 47, 43, 33]
row_labels = [36, 5, 47, 35, 17, 30, 21,49, 45]

In [24]:
# 3 functions.
# - one to create possible grids
# - one to solve the puzzle for a given set of "L"s
# - one to print showing the "L"s

def setup_grid(alignments):
    y = np.array([[1]])
    for i in range(0,8):
        temp = np.ones((i+2,i+2),dtype=int)*(i+2)
        if alignments[i] == 0:         
            temp[:-1,:-1] = y
        if alignments[i] == 1:         
            temp[1:,:-1] = y
        if alignments[i] == 2:         
            temp[:-1,1:] = y
        if alignments[i] == 3:         
            temp[1:,1:] = y                  
        y=temp
    return y

    
def solve_matrix(y,col_labels,row_labels):
    nums = range(1, 10)
    problem = pulp.LpProblem('Problem') 
    x = pulp.LpVariable.dicts('x', [(row, col) for row in nums for col in nums],lowBound=0,upBound=1, cat='Binary') # declare decision variables

    for index in nums:
        mask = (y == index)
        problem += pulp.lpSum([x[(row, col)] * mask[row-1,col-1] for row in range(1, 10) for col in range(1,10)]) == index

    for row in nums:
        problem += pulp.lpSum([y[row-1, col-1] * x[(row, col)] for col in nums]) == col_labels[row - 1]

    for col in nums:
        problem += pulp.lpSum([y[row-1, col-1] * x[(row, col)] for row in nums]) == row_labels[col - 1]

    #Solve LP
    problem.solve()

    solution = np.zeros((9,9),dtype=int)
    for row in nums:
        for col in nums:
            solution[row - 1][col - 1] = x[(row, col)].varValue

    return np.array(solution) , y

def sol_print(solved,matrix):
    fig,ax = plt.subplots(1,1,figsize=(5,5))
    ax = sns.heatmap(matrix,annot=solved*matrix,cbar=False,cmap="YlGnBu")
    ax.axis("off")

In [ ]:
start =  time.perf_counter()

for a,b,c,d,e,f,g,h in itertools.product(range(4),range(4),range(4),range(4),range(4),range(4),range(4),range(4)):
    arrangment = [a,b,c,d,e,f,g,h]
    y = setup_grid(arrangment)
    solved,matrix = solve_matrix(y,col_labels,row_labels)
    if np.sum(np.abs(np.sum(solved*matrix,axis=0)-row_labels)) + np.sum(np.abs(np.sum(solved*matrix,axis=1)-col_labels)) == 0:
        print("solved with arrangement",arrangment)
        break                                                                                                                                                                  

stop =  time.perf_counter()
print('Solution took {:0.4f} seconds\n'.format((stop-start)))
sol_print(solved,matrix)
print("row check",np.sum(solved*matrix,axis=0)-row_labels)
print("col check",np.sum(solved*matrix,axis=1)-col_labels)
print("\n")

### Puzzle solution
<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2016/06/may16_solution.png" width="500" height="600">

In [ ]:
###test with correct alignment

start =  time.perf_counter()

alignments = [2,1,3,3,2,0,0,1]

y = setup_grid(alignments)
solved,matrix = solve_matrix(y,col_labels,row_labels)

stop =  time.perf_counter()
print('Solution took {:0.4f} seconds\n'.format((stop-start)))
sol_print(solved,matrix)
print("row check",np.sum(solved*matrix,axis=0)-row_labels)
print("col check",np.sum(solved*matrix,axis=1)-col_labels)
print("\n")